In [ ]:
# Importing the required Python libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt

import lstm_model_modules as lmm

In [ ]:
# Calling the above function on the csv file containing the training data
training_file_path = 'train_data.csv'
data = lmm.preprocess_csv_file(training_file_path)

In [ ]:
# Getting the scaling factor
scaler = lmm.return_scaler(training_file_path)

# Scaling only the demand feature in the Pandas dataframe that we got above 
data['Demand'] *= scaler

In [ ]:
# Determining the train dataset length as 80% of the length of the above dataframe
train_size = int(len(data) * 0.8)

# Splitting the above dataset into train and validation sets:
train_data = data[:train_size]
val_data = data[train_size:]

In [ ]:
# Preprocessing the train and validation sets, using a specific lookback range (n_past)
# and a specific number of values that we want to predict in the future (n_future)
n_future = 1   
n_past = 336
X_train, Y_train = lmm.create_dataset_updated(train_data, n_future, n_past)
X_val, Y_val = lmm.create_dataset_updated(val_data, n_future, n_past)

In [ ]:
# Building the LSTM model
model = Sequential()
# model.add(Bidirectional(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False)))
model.add(LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
# model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1]))

# Compiling the model
model.compile(optimizer='adam', loss='mae')

In [ ]:
# Defining early stopping for the model
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Fitting the model using early stopping
history = model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_val, Y_val), callbacks=[es])

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
# Saving the best model
model.save("multivariate_lstm_model.h5")